In [1]:
Packages <- c("dplyr","nleqslv", "broom","cubature", "geosphere", "data.table",  "ggplot2", "bbmle", "stringr",  "lubridate", "RColorBrewer")

invisible(suppressPackageStartupMessages(lapply(Packages, library, character.only = TRUE)))

setwd('/local/home/katrinac/oceanography')
"%!in%" <- function(x,table) match(x,table, nomatch = 0) == 0
source("~/parentage/kernel_fitting/1340_loci/functions/ll_kt_both_bbmle.R")
source("~/parentage/kernel_fitting/1340_loci/functions/GenGausKernInt_sum0.5.R") #integrate_kernel_sum1
source("~/parentage/kernel_fitting/1340_loci/functions/GenGausKernInt_sum1.R") #integrate_kernel_sum0.5
source("~/parentage/kernel_fitting/1340_loci/functions/cdf_solve.R") #median
source("~/parentage/kernel_fitting/1340_loci/functions/cdf_solve90.R") #dist 90% retained

#read in the kernel fitting summary
kernels <- fread(file="~/parentage/kernel_fitting/1340_loci/final_results/tables/kernel_fitting_summary.csv")
#read in the centroids adjusted for the simulation, so the Magbangons combined 
centroids <- fread(file="~/oceanography/script_output/SurveyData/SimulationCentroids.csv")
#read in the distance matrix adjusted for the simulation
distances <- fread(file="~/oceanography/script_output/SurveyData/SimulationAllDistances.csv")
#read in the table with number of recruits sampled at each site for each year
AnnualRecsSamp <- fread(file="~/oceanography/script_output/SurveyData/AnnualRecruitsSampled.csv")
#read in the table of the proportion of anemones sampled at each site for each year
PropSamp <- fread(file="~/oceanography/script_output/SurveyData/ProportionHabitatSampled.csv")

#read in the ROMS simulation connectivity table
SimConn <- fread(file="~/oceanography/script_output/SimulationSummaryTables/SimConnectivityTable.csv")

#make vectors defining sites we didn't sample, but that are in the model, and the sandflats specifically 
UnsampledSites <- c("SF1", "SF2", "SF3", "SF4", "SF5", "SF6", "Pangasugan", "Other", "CAI") 
SandFlats <- c("SF1", "SF2", "SF3", "SF4", "SF5", "SF6") 

In [2]:
#make the connectivity data long form
SimConn <- SimConn[destination !="Other" & source %!in% SandFlats][ #we don't care about "other" as a destination, and sand flats can't be a source
    ,!c("SourceSampled", "DestSampled")]#drop these columns, it was used for sorting in the process_ROMS script but at this point it's confusing
check1 <- sum(SimConn$DailyParticles)   
#setkey(SimConn, source)
SimConn <- SimConn[rep(seq_len(nrow(SimConn)), SimConn$DailyParticles), 1:ncol(SimConn)][ #make long form, repeat each row the number of times specified in "DailyParticles"
    , ParticleID := paste ("P", .I, sep = "", collapse = NULL)]#finally, make a new column that's the particle ID for each row

nrow(SimConn)==check1 #should be TRUE, meaning the correct number of particles are retained

#the below code is subsampling particles, but we want to do that within the loop of kernel fitting
#SimConnDTDir3 <- SimConnDTDir2[c(SimConnDTDir2[, .I[source != "Other"]], sample(SimConnDTDir2[, .I[source == "Other"]], length(SimConnDTDir2[, .I[source == "Other"]])*.1))]
#SimConnDTDir3 <- SimConnDTDir2[c(SimConnDTDir3[, .I[source != "CAI"]], sample(SimConnDTDir3[, .I[source == "CAI"]], length(SimConnDTDir3[, .I[source == "CAI"]])*.1))]


[1] TRUE

In [3]:
SimConn[, .(.N), by = .(source)]

source,N
Other,51656092
CAI,159712708
Haina,25118
Tamakin Dacot,226012
Gabas,171252
Visca,81160
Pangasugan,70481
Poroc Rose,84056
Poroc San Flower,94236
San Agustin,82882


In [4]:
#make a table with the survey information for each site (how many fish sampled, prop anems sampled, total number of anems at site)
SurveyData <- AnnualRecsSamp[PropSamp, on=.(year=end_year, site)][#join the sampling tables together
    is.na(n_offs_gen), n_offs_gen := 0][#change NA's to 0
    ,-"time_frame"]#drop the time_frame column, we can key with end_year
#setkey(SurveyData, site)


In [5]:
#join the simulation data with the survey data
source_join <- SurveyData[SimConn, on = .(site = source, year=YearSampled)]
setnames(source_join, c("site", "year", "n_offs_gen", "PropAnemSamp", "TotalAnems"), c("Source","YearSampled", "SourceNOffsAnnual",  "SourcePropSamp", "SourceTotalAnems"))

destination_join <- SurveyData[source_join, on = .( site= destination, year=YearSampled)]
setnames(destination_join, c("site", "year", "n_offs_gen", "PropAnemSamp", "TotalAnems"), c("Dest","YearSampled", "DestNOffsAnnual",  "DestPropSamp", "DestTotalAnems"))



In [6]:
#lets see if we can do all the kernel fitting in one loop! be ambitious! this table created here will be what we loop through for kernel fitting
SimDates <- destination_join[,.(date= unique(date)), by=.(YearSampled)][ #make a data.table of dates in that year-
    , DateSeqLin := ((0.25/2)/5)*.I, by=.(YearSampled)][ #then add a column of row number mulitplied by the linear survival model slope
    , SurvWeight := DateSeqLin/max(DateSeqLin), by=.(YearSampled)][ #then add a column normalizing that as the survival probability, BUT if not eliminating new recruits (months 4/5) multiply the denominator by 1.25 so that nothing is an arbitrary prob=1
    ,-"DateSeqLin"]

#check this looks right- 2014 has fewer dates than the other years because the simulation is truncated for that time, and should see the SurvWeight column get low again when the year changes
#SimDates[, .(.N), by = .(YearSampled)]
#SimDates[240:250,]


#now join with the simulation data table
date_join <- SimDates[destination_join, on=.(date, YearSampled)][#joining with YearSampled too is redundant, can otherwise drop the column from SimDates before joining #
    SimMonth != c(4,5)] #filter out particles recruiting in April/May, they would be new recruits
 
nrow(date_join)==nrow(SimConn[SimMonth != c(4,5)]) #should be TRUE

#finally, weight the sources by their number of anemones relative to the total number of anemones across the surveyed region
total_anems <- sum(SurveyData[year=="2014"][,"TotalAnems"]) #get the total number of anems recorded for the simulation time period
date_join <- date_join[, SourceSize := SourceTotalAnems/total_anems, by=.(Source)]

#first thing is sampling the simulation data like we sampled for the survey data

#outside of the loop, trim this to only be the destinations we sampled, and 10% of the other/camotes particles (so the loop is faster). After finishing this code, we're going to figure out the most representative proportion to sample*

dest_sampled <- date_join[DestPropSamp >0]
check1 <- nrow(dest_sampled)
dest_sampled <- dest_sampled[c(dest_sampled[, .I[Source != "Other"]], sample(dest_sampled[, .I[Source == "Other"]], length(dest_sampled[, .I[Source == "Other"]])*.1))]
check2 <- nrow(dest_sampled)
dest_sampled <- dest_sampled[c(dest_sampled[, .I[Source != "CAI"]], sample(dest_sampled[, .I[Source == "CAI"]], length(dest_sampled[, .I[Source == "CAI"]])*.1))]
check3 <- nrow(dest_sampled)

#check that we have less rows, should both be TRUE
check1 > check2
check2 > check3

#join in the number of parentage matches observed by year
dest_sampled <- kernels[Year %in% c("2012", "2013", "2014")][, Year:=as.integer(Year)][,c("Year", "NumParentageMatches")][dest_sampled, on=.(Year=YearSampled)]#[
    #, ParticleID := paste ("P", .I, sep = "", collapse = NULL)]#finally, make a new column that's the particle ID for each row

#setkey(dest_sampled, "Year", "Dest", "Source")




[1] TRUE

[1] TRUE

[1] TRUE

In [10]:
nrow(dest_sampled)
nrow(SimConn)

[1] 363760

[1] 212861852

In [7]:
#save inter file
#fwrite(dest_sampled, file="~/oceanography/script_output/LongFormConnWithProbsTest.csv")

In [65]:
#make an empty data.table to hold all of the simulation results
row_num <- 4*1000
SimKernels <- data.table(year=integer(), k=numeric(), theta=numeric(), mdd=numeric(), dist90=numeric(), iteration=numeric())[1:row_num]

#sample the particle data
sim_sample <- dest_sampled[, .SD[sample(.N, DestNOffsAnnual, prob=SurvWeight)], by = c("Year", "Dest")] #randomly sample rows (particles) from the table according to the survival weighting, based on the number we sampled at each site in each year of surveys
check1 <- nrow(sim_sample)

#assign parentage
sim_parentage <- sim_sample[SourcePropSamp > 0][, .SD[sample(.N, NumParentageMatches, prob=SourcePropSamp*SourceSize)], by = .(Year)][#now randomly assign parentage or not parentage, based on how well we sampled the source and the number of parentage matches we had in that year
                , Parentage := 1]

#for faster searching, set keys
setkey(sim_parentage, ParticleID)
setkey(sim_sample, ParticleID)

sim_sample <- rbind(sim_sample[ParticleID %!in% sim_parentage$ParticleID][, Parentage := 0], sim_parentage)[, c("Year", "Source", "Dest", "Parentage")] #add back in to the unassigned particles, select only the columns necessary

#check results
nrow(sim_sample)==check1 #should be TRUE
sum(sim_sample$Parentage) #should be 37


[1] TRUE

[1] 37

year,site,n_offs_gen,PropAnemSamp,TotalAnems
2012,Cabatoan,4,0.4230769,26
2012,Caridad Cemetery,0,0.0000000,4
2012,Caridad Proper,0,0.0000000,4
2012,Elementary School,0,0.0000000,7
2012,Gabas,0,0.0000000,9
2012,Haina,0,0.0000000,104


Year,Source,Dest,Parentage
2012,Wangag,Wangag,0
2012,Sitio Baybayon,Cabatoan,0
2012,Elementary School,Wangag,0
2012,Hicgop South,Magbangon,0
2013,Other,Visca,0
2013,Other,Magbangon,0


In [55]:
#outside of the loop
#each year will require a different set of survey data, so make a list of each and index by site for fast look up
PropSampTable <- SurveyData[PropAnemSamp >0, c("year", "site")][
    , c("Year", "Source", "Dest", "Parentage")] #only grab the columns we need
setkey(year)

#make sure all sampled sites are represented by joining the survey data to the sampled simulation
PropSampTable <- SurveyData[PropAnemSamp >0, c("year", "site")][, .(Source=site, Dest=site, Year=year)] #will join to the simulated sampling table by source and dest, so make those each a column from site and preserve the year variable as a key
unq_survey <- unique(PropSampTable, by=c("Source", "Dest", "Year"))


In [61]:
#within the loop
sim_sample <- PropSampTable[sim_sample, on=.(Year, Source)]
#check all is well-
#sum(sim_sample$Parentage) #should be 37

#add in the routes we could have assigned given our sampling so the parentage matrix is complete
unq_sim_sample <- unique(sim_sample, by=c("Source", "Dest", "Year"))

add_routes <- unq_survey[!unq_sim_sample, on = names(unq_survey)][ #what combos are not appearing because we didn't sample particles, but the route is possible based on our survey sampling
    , Parentage := 0] #add the parentage column 

#add back into the sampled simulation data
sim_sample <- rbind(sim_sample, add_routes)
#check all is well-
#sum(sim_sample$Parentage) #should be 37

[1] 37

In [90]:
#make summary table that will be transformed into the matrix
sim_sample_summary <- sim_sample[, .(.N), by= c("Source", "Dest", "Year", "Parentage")]

In [152]:
assigned <- sim_sample[Parentage==1, !c("Parentage", "Year")][, num_assigned := .(.N), by= c("Source", "Dest")] 
#unassigned <- sim_sample_summary[Parentage==0, !c("Parentage", "Year")] 

assigned

In [154]:
sum(assigned$num_assigned)

[1] 123

In [144]:
mat <- dcast(sim_sample, Source ~ Dest, value.var="Parentage", fun.aggregate = sum)[, -"Source"]
sum(mat[,2:ncol(mat)])

unassigned <- sim_sample[Parentage==0, .(.N), by= "Dest"][order(Dest),N] 
#full_mat <- rbind(mat, t(unassigned), use.names=F)

[1] 34

In [145]:
sum(unassigned)

[1] 455

In [114]:
#make a parentage matrix for the assigned
assigned_mat <- dcast(assigned, Source+Parentage ~ Dest, value.var="N", fun.aggregate = sum)
sum(assigned_mat[,2:ncol(assigned_mat)])

[1] 37

In [ ]:
  #NOW FORMAT INTO PARENTAGE MATRIX
    
                PropSamp2012 <- PropSamp %>%  #add in all sampled sites as possible parent sites
                            filter(end_year =="2012" & PropSamp > 0) %>%
                            #rename(source="site") %>%
                            select(-end_year, -PropSamp)

                #PropSamp2012$destination <- PropSamp2012$source #make another column for destination
                
                #make a table with the total number of particles sampled at each site
                TotalSimSamp <- SimulatedSampling2012Par3 %>% 
                            select(source, destination) %>%
                            group_by(destination) %>%
                            summarise(NumSimSampRec=n()) %>%
                            ungroup()
               
                
                #add in all surveyed sites
                TotalSimSamp2 <- left_join(PropSamp2012, TotalSimSamp, by=c(site="destination")) %>% #join sites to destination, which will add a row for the surveyed sites with no samples
                    mutate(NumSimSampRec=ifelse(is.na(NumSimSampRec), 0, NumSimSampRec)) %>% #change the Na's that result to 0s, because we didn't get samples there
                    rename(destination="site")#rename sites to destination after joining
                
                
                
                #make a table with the total number of parentage matches found in the simulated loop
                TotalSimPar <- SimulatedSampling2012Par3 %>% 
                            select(source, destination, Parentage) %>%
                            group_by(source,destination) %>%
                            mutate(NumPar=sum(Parentage)) %>% #sum all the parentage observations along each ROUTE
                            select(-Parentage) %>%
                            ungroup() %>%
                            distinct(source, destination, .keep_all = T)
                
                
                #combine for a summary table BY DESTINATION, this below will be for UNASSIGNED ROWS
                SimSampSummary2012 <- left_join(TotalSimSamp2, TotalSimPar, by="destination") %>%
                    distinct(source, destination, .keep_all = T) %>%
                    group_by(destination) %>%
                    mutate(NumPar=ifelse(is.na(NumPar), 0, NumPar)) %>% #change the Na's that result to 0s, because we didn't get samples there
                    mutate(NumPar=sum(NumPar)) %>% #sum all the parentage observations at each DESTINATION
                    distinct(destination, .keep_all = T) %>%
                    select(-source) %>%
                    mutate(NumUnassigned=NumSimSampRec-NumPar)
                                
                
                #finally, join all of the sites surveyed for a complete parentage matrix
                #for this, need to have a source/destination column for all sites
                PropSamp2012 <- PropSamp2012 %>%
                                rename(source="site")
                
                PropSamp2012$destination <- PropSamp2012$source #make another column for destination
                
                #this below will become parentage matrix! 
                TotalSimPar2 <- full_join(TotalSimPar,PropSamp2012, by=c("source", "destination")) %>%
                    mutate(NumPar=ifelse(is.na(NumPar), 0, NumPar))
        
                
                #spread into matrix format
                SimDispMat <- TotalSimPar2 %>%
                    filter(source %in% NGenOffs2012$site) %>%
                    arrange(source, destination)  %>%
                    spread(destination, NumPar) %>%
                    select(-source)
                SimDispMat[is.na(SimDispMat)] <- 0
                
                #add in unassigned row
                unassigned <- t(as.matrix(SimSampSummary2012 %>% arrange(destination) %>% ungroup() %>% select(NumUnassigned)))
                rownames(unassigned)<-NULL
                colnames(unassigned)<-names(SimDispMat)
                
                SimDispMatFull2012 <- rbind(SimDispMat, unassigned)
                colnames(SimDispMatFull2012) <- NULL

        #The full remade parentage matrix
        Assignments2012 <- SimDispMatFull2012


__Get all the components of the kernel fitting process together__

In [ ]:

#awesome! everything works! let's rip with a 1000 iteration for loop
SimulatedKernels <- data.table(year=integer(), k=numeric(), theta=numeric(), mdd=numeric(), dist90=numeric(), iteration=numeric())

#don't print warnings for this loop, they are only for setting row names in a tibble which is depracated. But it works. 
options(warn=-1)

pb <- txtProgressBar(min = 0, max = 1000, style = 3)

StartTime <- Sys.time()

for(n in 1:1000){
        
    #SAMPLE THE SIMULATION DATA, then fit a kernel
    
    #make an empty dataframe with the same columns as original, this will be sampled particles
    SimulatedSampling2012 = Simulation2012[FALSE,]
    
    for(i in 1:nrow(NGenOffs2012)){ 
        
        destination_eval <- as.character(NGenOffs2012$site[i]) #pick out a destination site
        
        SimSampDestination <- Simulation2012Surv %>% 
                                filter(destination==destination_eval) %>%
                                sample_n(as.numeric(NGenOffs2012$n_offs_gen[i]), weight=SurvivalWeightLin, replace=F) %>% #sample particles that landed at that destination, number corresponding to the actual number sampled at that site in this year
                                select(one_of(names(Simulation2012)))
        
        SimulatedSampling2012 <- bind_rows(SimulatedSampling2012, SimSampDestination) #build into a sampled particle df
    }
    
    #assign a numeric ID for each row (which is a sampled particle)
    SimulatedSampling2012 <- SimulatedSampling2012 %>%
        mutate(ParticleSampID=paste ("P", row_number(), sep = "", collapse = NULL))
    
    #nrow(SimulatedSampling2012)==sum(NGenOffs2012$n_offs_gen) #should be TRUE
    
    #now randomly assign parentage match status to some of these rows
    NumPar <- as.numeric(kernels %>%
        filter(Year=="2012") %>%
        select(NumParentageMatches))
    
    SimulatedSampling2012Par <- SimulatedSampling2012 %>%
        ungroup() %>%
        filter(source %in% NGenOffs2012$site) %>% #we can only assign parentage if we sampled the source site, we've already ensured that we sampled the destination site in the for loop by indexing destination sites to evaluate 
        sample_n(NumPar, replace=F) %>% #sample number of rows consitent with numbers of parentage matches
        mutate(Parentage=1) #%>% #assign these rows a positive match value
        #arrange(date, source, destination) %>%#arrange by the same order as the orginal df, then bind columns together to avoid unexpected repeats from a join
        #select(date, source, destination, Parentage) #drop columns that will result in repeats 
    
    #find the rows that haven't been assigned parentage
    SimulatedSampling2012_2 <- anti_join((SimulatedSampling2012 %>% ungroup()), SimulatedSampling2012Par, by=c("ParticleSampID")) 
    
    SimulatedSampling2012_2$Parentage <- 0 #add in the column for parentage
    
    #combine parentage and non-parentage dfs
    SimulatedSampling2012Par3 <- bind_rows(SimulatedSampling2012Par, SimulatedSampling2012_2) %>%
        mutate(YearSampled=as.character(YearSampled))
    
    
    #NOW FORMAT INTO PARENTAGE MATRIX
    
                PropSamp2012 <- PropSamp %>%  #add in all sampled sites as possible parent sites
                            filter(end_year =="2012" & PropSamp > 0) %>%
                            #rename(source="site") %>%
                            select(-end_year, -PropSamp)

                #PropSamp2012$destination <- PropSamp2012$source #make another column for destination
                
                #make a table with the total number of particles sampled at each site
                TotalSimSamp <- SimulatedSampling2012Par3 %>% 
                            select(source, destination) %>%
                            group_by(destination) %>%
                            summarise(NumSimSampRec=n()) %>%
                            ungroup()
               
                
                #add in all surveyed sites
                TotalSimSamp2 <- left_join(PropSamp2012, TotalSimSamp, by=c(site="destination")) %>% #join sites to destination, which will add a row for the surveyed sites with no samples
                    mutate(NumSimSampRec=ifelse(is.na(NumSimSampRec), 0, NumSimSampRec)) %>% #change the Na's that result to 0s, because we didn't get samples there
                    rename(destination="site")#rename sites to destination after joining
                
                
                
                #make a table with the total number of parentage matches found in the simulated loop
                TotalSimPar <- SimulatedSampling2012Par3 %>% 
                            select(source, destination, Parentage) %>%
                            group_by(source,destination) %>%
                            mutate(NumPar=sum(Parentage)) %>% #sum all the parentage observations along each ROUTE
                            select(-Parentage) %>%
                            ungroup() %>%
                            distinct(source, destination, .keep_all = T)
                
                
                #combine for a summary table BY DESTINATION, this below will be for UNASSIGNED ROWS
                SimSampSummary2012 <- left_join(TotalSimSamp2, TotalSimPar, by="destination") %>%
                    distinct(source, destination, .keep_all = T) %>%
                    group_by(destination) %>%
                    mutate(NumPar=ifelse(is.na(NumPar), 0, NumPar)) %>% #change the Na's that result to 0s, because we didn't get samples there
                    mutate(NumPar=sum(NumPar)) %>% #sum all the parentage observations at each DESTINATION
                    distinct(destination, .keep_all = T) %>%
                    select(-source) %>%
                    mutate(NumUnassigned=NumSimSampRec-NumPar)
                                
                
                #finally, join all of the sites surveyed for a complete parentage matrix
                #for this, need to have a source/destination column for all sites
                PropSamp2012 <- PropSamp2012 %>%
                                rename(source="site")
                
                PropSamp2012$destination <- PropSamp2012$source #make another column for destination
                
                #this below will become parentage matrix! 
                TotalSimPar2 <- full_join(TotalSimPar,PropSamp2012, by=c("source", "destination")) %>%
                    mutate(NumPar=ifelse(is.na(NumPar), 0, NumPar))
        
                
                #spread into matrix format
                SimDispMat <- TotalSimPar2 %>%
                    filter(source %in% NGenOffs2012$site) %>%
                    arrange(source, destination)  %>%
                    spread(destination, NumPar) %>%
                    select(-source)
                SimDispMat[is.na(SimDispMat)] <- 0
                
                #add in unassigned row
                unassigned <- t(as.matrix(SimSampSummary2012 %>% arrange(destination) %>% ungroup() %>% select(NumUnassigned)))
                rownames(unassigned)<-NULL
                colnames(unassigned)<-names(SimDispMat)
                
                SimDispMatFull2012 <- rbind(SimDispMat, unassigned)
                colnames(SimDispMatFull2012) <- NULL

        #The full remade parentage matrix
        Assignments2012 <- SimDispMatFull2012
    
    #NOW FIT THE KERNEL


x <- list(Distances=Distances, Assignments=Assignments2012, Sampled_reefs=Sampled_reefs2012, Reef_sizes=Reef_sizes, Adult_sample_proportions=Adult_sample_proportions2012) #put inputs into a list because that's the bbmle format

Sim2012Fit <- suppressWarnings(mle2(LL_kt_bbmle, start=list(k=-3, theta=1), lower=c(-10, 0.15), upper=c(10, 8), method="L-BFGS-B", data=x, control=list(maxit=500)))

BestK2012 <- as.numeric(coef(Sim2012Fit)[1])
BestTheta2012 <- as.numeric(coef(Sim2012Fit)[2])
MDD2012 <- as.numeric(cubintegrate(integrate_kernel_sum1, lower = 0, upper = Inf, k=BestK2012, theta=BestTheta2012, , method = "pcubature")$integral)
k_eval <- BestK2012
theta_eval <- BestTheta2012
Med2012  <- round(nleqslv(x = 7, fn = cdf_solve)$x, 2) 
Dist90_2012 <- round(nleqslv(x = 7, fn = cdf_solve90)$x, 2)
    
    #store the info in this df
    SimulatedKernels2012_beta <- as.data.frame(matrix(nrow=1, ncol=7), stringsAsFactors = FALSE)
    colnames(SimulatedKernels2012_beta) <- col
    
    SimulatedKernels2012_beta$year <- 2012
    SimulatedKernels2012_beta$k <- BestK2012
    SimulatedKernels2012_beta$theta <- BestTheta2012
    SimulatedKernels2012_beta$mdd <- MDD2012
    SimulatedKernels2012_beta$med <- Med2012
    SimulatedKernels2012_beta$dist90 <- Dist90_2012
    SimulatedKernels2012_beta$iteration <- n
    
    
#join results into larger df
SimulatedKernels2012 <- bind_rows(SimulatedKernels2012, SimulatedKernels2012_beta)
    
setTxtProgressBar(pb, n)


}
close(pb)
EndTime <- Sys.time()
EndTime-StartTime
options(warn=0) #turn warnings back on


write.csv(SimulatedKernels2012, file="~/oceanography/script_output/KernelFits/Adjust_Survival_NoRecentRecruits/1000SimulatedKernels2012SurvNoRecentRec.csv", row.names=F)
SimulatedKernels2013 <- as.data.frame(matrix(nrow=0, ncol=7), stringsAsFactors = FALSE)
colnames(SimulatedKernels2013) <- col

#don't print warnings for this loop, they are only for setting row names in a tibble which is depracated. But it works. 
options(warn=-1)

pb <- txtProgressBar(min = 0, max = 1000, style = 3)

StartTime <- Sys.time()

for(n in 1:1000){
        
    #SAMPLE THE SIMULATION DATA, then fit a kernel
    
    #make an empty dataframe with the same columns as original, this will be sampled particles
    SimulatedSampling2013 = Simulation2013[FALSE,]
    
    for(i in 1:nrow(NGenOffs2013)){ 
        
        destination_eval <- as.character(NGenOffs2013$site[i]) #pick out a destination site
        
        SimSampDestination <- Simulation2013Surv %>% 
                                filter(destination==destination_eval) %>%
                                sample_n(as.numeric(NGenOffs2013$n_offs_gen[i]), weight=SurvivalWeightLin, replace=F) %>% #sample particles that landed at that destination, number corresponding to the actual number sampled at that site in this year
                                select(one_of(names(Simulation2013)))
        
        SimulatedSampling2013 <- bind_rows(SimulatedSampling2013, SimSampDestination) #build into a sampled particle df
    }
    
    #assign a numeric ID for each row (which is a sampled particle)
    SimulatedSampling2013 <- SimulatedSampling2013 %>%
        mutate(ParticleSampID=paste ("P", row_number(), sep = "", collapse = NULL))
    
    #nrow(SimulatedSampling2013)==sum(NGenOffs2013$n_offs_gen) #should be TRUE
    
    #now randomly assign parentage match status to some of these rows
    NumPar <- as.numeric(kernels %>%
        filter(Year=="2013") %>%
        select(NumParentageMatches))
    
    SimulatedSampling2013Par <- SimulatedSampling2013 %>%
        ungroup() %>%
        filter(source %in% NGenOffs2013$site) %>% #we can only assign parentage if we sampled the source site, we've already ensured that we sampled the destination site in the for loop by indexing destination sites to evaluate 
        sample_n(NumPar, replace=F) %>% #sample number of rows consitent with numbers of parentage matches
        mutate(Parentage=1) #%>% #assign these rows a positive match value
        #arrange(date, source, destination) %>%#arrange by the same order as the orginal df, then bind columns together to avoid unexpected repeats from a join
        #select(date, source, destination, Parentage) #drop columns that will result in repeats 
    
    #find the rows that haven't been assigned parentage
    SimulatedSampling2013_2 <- anti_join((SimulatedSampling2013 %>% ungroup()), SimulatedSampling2013Par, by=c("ParticleSampID")) 
    
    SimulatedSampling2013_2$Parentage <- 0 #add in the column for parentage
    
    #combine parentage and non-parentage dfs
    SimulatedSampling2013Par3 <- bind_rows(SimulatedSampling2013Par, SimulatedSampling2013_2) %>%
        mutate(YearSampled=as.character(YearSampled))
    
    
    #NOW FORMAT INTO PARENTAGE MATRIX
    
                PropSamp2013 <- PropSamp %>%  #add in all sampled sites as possible parent sites
                            filter(end_year =="2013" & PropSamp > 0) %>%
                            #rename(source="site") %>%
                            select(-end_year, -PropSamp)

                #PropSamp2013$destination <- PropSamp2013$source #make another column for destination
                
                #make a table with the total number of particles sampled at each site
                TotalSimSamp <- SimulatedSampling2013Par3 %>% 
                            select(source, destination) %>%
                            group_by(destination) %>%
                            summarise(NumSimSampRec=n()) %>%
                            ungroup()
               
                
                #add in all surveyed sites
                TotalSimSamp2 <- left_join(PropSamp2013, TotalSimSamp, by=c(site="destination")) %>% #join sites to destination, which will add a row for the surveyed sites with no samples
                    mutate(NumSimSampRec=ifelse(is.na(NumSimSampRec), 0, NumSimSampRec)) %>% #change the Na's that result to 0s, because we didn't get samples there
                    rename(destination="site")#rename sites to destination after joining
                
                
                
                #make a table with the total number of parentage matches found in the simulated loop
                TotalSimPar <- SimulatedSampling2013Par3 %>% 
                            select(source, destination, Parentage) %>%
                            group_by(source,destination) %>%
                            mutate(NumPar=sum(Parentage)) %>% #sum all the parentage observations along each ROUTE
                            select(-Parentage) %>%
                            ungroup() %>%
                            distinct(source, destination, .keep_all = T)
                
                
                #combine for a summary table BY DESTINATION, this below will be for UNASSIGNED ROWS
                SimSampSummary2013 <- left_join(TotalSimSamp2, TotalSimPar, by="destination") %>%
                    distinct(source, destination, .keep_all = T) %>%
                    group_by(destination) %>%
                    mutate(NumPar=ifelse(is.na(NumPar), 0, NumPar)) %>% #change the Na's that result to 0s, because we didn't get samples there
                    mutate(NumPar=sum(NumPar)) %>% #sum all the parentage observations at each DESTINATION
                    distinct(destination, .keep_all = T) %>%
                    select(-source) %>%
                    mutate(NumUnassigned=NumSimSampRec-NumPar)
                                
                
                #finally, join all of the sites surveyed for a complete parentage matrix
                #for this, need to have a source/destination column for all sites
                PropSamp2013 <- PropSamp2013 %>%
                                rename(source="site")
                
                PropSamp2013$destination <- PropSamp2013$source #make another column for destination
                
                #this below will become parentage matrix! 
                TotalSimPar2 <- full_join(TotalSimPar,PropSamp2013, by=c("source", "destination")) %>%
                    mutate(NumPar=ifelse(is.na(NumPar), 0, NumPar))
        
                
                #spread into matrix format
                SimDispMat <- TotalSimPar2 %>%
                    filter(source %in% NGenOffs2013$site) %>%
                    arrange(source, destination)  %>%
                    spread(destination, NumPar) %>%
                    select(-source)
                SimDispMat[is.na(SimDispMat)] <- 0
                
                #add in unassigned row
                unassigned <- t(as.matrix(SimSampSummary2013 %>% arrange(destination) %>% ungroup() %>% select(NumUnassigned)))
                rownames(unassigned)<-NULL
                colnames(unassigned)<-names(SimDispMat)
                
                SimDispMatFull2013 <- rbind(SimDispMat, unassigned)
                colnames(SimDispMatFull2013) <- NULL

        #The full remade parentage matrix
        Assignments2013 <- SimDispMatFull2013
    
    #NOW FIT THE KERNEL


x <- list(Distances=Distances, Assignments=Assignments2013, Sampled_reefs=Sampled_reefs2013, Reef_sizes=Reef_sizes, Adult_sample_proportions=Adult_sample_proportions2013) #put inputs into a list because that's the bbmle format

Sim2013Fit <- suppressWarnings(mle2(LL_kt_bbmle, start=list(k=-3, theta=1), lower=c(-10, 0.15), upper=c(10, 8), method="L-BFGS-B", data=x, control=list(maxit=500)))

BestK2013 <- as.numeric(coef(Sim2013Fit)[1])
BestTheta2013 <- as.numeric(coef(Sim2013Fit)[2])
MDD2013 <- as.numeric(cubintegrate(integrate_kernel_sum1, lower = 0, upper = Inf, k=BestK2013, theta=BestTheta2013, , method = "pcubature")$integral)
k_eval <- BestK2013
theta_eval <- BestTheta2013
Med2013  <- round(nleqslv(x = 7, fn = cdf_solve)$x, 2) 
Dist90_2013 <- round(nleqslv(x = 7, fn = cdf_solve90)$x, 2)
    
    #store the info in this df
    SimulatedKernels2013_beta <- as.data.frame(matrix(nrow=1, ncol=7), stringsAsFactors = FALSE)
    colnames(SimulatedKernels2013_beta) <- col
    
    SimulatedKernels2013_beta$year <- 2013
    SimulatedKernels2013_beta$k <- BestK2013
    SimulatedKernels2013_beta$theta <- BestTheta2013
    SimulatedKernels2013_beta$mdd <- MDD2013
    SimulatedKernels2013_beta$med <- Med2013
    SimulatedKernels2013_beta$dist90 <- Dist90_2013
    SimulatedKernels2013_beta$iteration <- n
    
    
#join results into larger df
SimulatedKernels2013 <- bind_rows(SimulatedKernels2013, SimulatedKernels2013_beta)
    
setTxtProgressBar(pb, n)


}
close(pb)
EndTime <- Sys.time()
EndTime-StartTime
options(warn=0) #turn warnings back on


write.csv(SimulatedKernels2013, file="~/oceanography/script_output/KernelFits/Adjust_Survival_NoRecentRecruits/1000SimulatedKernels2013SurvNoRecentRec.csv", row.names=F)

SimulatedKernels2014 <- as.data.frame(matrix(nrow=0, ncol=7), stringsAsFactors = FALSE)
colnames(SimulatedKernels2014) <- col

#don't print warnings for this loop, they are only for setting row names in a tibble which is depracated. But it works. 
options(warn=-1)

pb <- txtProgressBar(min = 0, max = 1000, style = 3)

StartTime <- Sys.time()

for(n in 1:1000){
        
    #SAMPLE THE SIMULATION DATA, then fit a kernel
    
    #make an empty dataframe with the same columns as original, this will be sampled particles
    SimulatedSampling2014 = Simulation2014[FALSE,]
    
    for(i in 1:nrow(NGenOffs2014)){ 
        
        destination_eval <- as.character(NGenOffs2014$site[i]) #pick out a destination site
        
        SimSampDestination <- Simulation2014Surv %>% 
                                filter(destination==destination_eval) %>%
                                sample_n(as.numeric(NGenOffs2014$n_offs_gen[i]), weight=SurvivalWeightLin, replace=F) %>% #sample particles that landed at that destination, number corresponding to the actual number sampled at that site in this year
                                select(one_of(names(Simulation2014)))  
        
        SimulatedSampling2014 <- bind_rows(SimulatedSampling2014, SimSampDestination) #build into a sampled particle df
    }
    
    #assign a numeric ID for each row (which is a sampled particle)
    SimulatedSampling2014 <- SimulatedSampling2014 %>%
        mutate(ParticleSampID=paste ("P", row_number(), sep = "", collapse = NULL))
    
    #nrow(SimulatedSampling2014)==sum(NGenOffs2014$n_offs_gen) #should be TRUE
    
    #now randomly assign parentage match status to some of these rows
    NumPar <- as.numeric(kernels %>%
        filter(Year=="2014") %>%
        select(NumParentageMatches))
    
    SimulatedSampling2014Par <- SimulatedSampling2014 %>%
        ungroup() %>%
        filter(source %in% NGenOffs2014$site) %>% #we can only assign parentage if we sampled the source site, we've already ensured that we sampled the destination site in the for loop by indexing destination sites to evaluate 
        sample_n(NumPar, replace=F) %>% #sample number of rows consitent with numbers of parentage matches
        mutate(Parentage=1) #%>% #assign these rows a positive match value
        #arrange(date, source, destination) %>%#arrange by the same order as the orginal df, then bind columns together to avoid unexpected repeats from a join
        #select(date, source, destination, Parentage) #drop columns that will result in repeats 
    
    #find the rows that haven't been assigned parentage
    SimulatedSampling2014_2 <- anti_join((SimulatedSampling2014 %>% ungroup()), SimulatedSampling2014Par, by=c("ParticleSampID")) 
    
    SimulatedSampling2014_2$Parentage <- 0 #add in the column for parentage
    
    #combine parentage and non-parentage dfs
    SimulatedSampling2014Par3 <- bind_rows(SimulatedSampling2014Par, SimulatedSampling2014_2) %>%
        mutate(YearSampled=as.character(YearSampled))
    
    
    #NOW FORMAT INTO PARENTAGE MATRIX
    
                PropSamp2014 <- PropSamp %>%  #add in all sampled sites as possible parent sites
                            filter(end_year =="2014" & PropSamp > 0) %>%
                            #rename(source="site") %>%
                            select(-end_year, -PropSamp)

                #PropSamp2014$destination <- PropSamp2014$source #make another column for destination
                
                #make a table with the total number of particles sampled at each site
                TotalSimSamp <- SimulatedSampling2014Par3 %>% 
                            select(source, destination) %>%
                            group_by(destination) %>%
                            summarise(NumSimSampRec=n()) %>%
                            ungroup()
               
                
                #add in all surveyed sites
                TotalSimSamp2 <- left_join(PropSamp2014, TotalSimSamp, by=c(site="destination")) %>% #join sites to destination, which will add a row for the surveyed sites with no samples
                    mutate(NumSimSampRec=ifelse(is.na(NumSimSampRec), 0, NumSimSampRec)) %>% #change the Na's that result to 0s, because we didn't get samples there
                    rename(destination="site")#rename sites to destination after joining
                
                
                
                #make a table with the total number of parentage matches found in the simulated loop
                TotalSimPar <- SimulatedSampling2014Par3 %>% 
                            select(source, destination, Parentage) %>%
                            group_by(source,destination) %>%
                            mutate(NumPar=sum(Parentage)) %>% #sum all the parentage observations along each ROUTE
                            select(-Parentage) %>%
                            ungroup() %>%
                            distinct(source, destination, .keep_all = T)
                
                
                #combine for a summary table BY DESTINATION, this below will be for UNASSIGNED ROWS
                SimSampSummary2014 <- left_join(TotalSimSamp2, TotalSimPar, by="destination") %>%
                    distinct(source, destination, .keep_all = T) %>%
                    group_by(destination) %>%
                    mutate(NumPar=ifelse(is.na(NumPar), 0, NumPar)) %>% #change the Na's that result to 0s, because we didn't get samples there
                    mutate(NumPar=sum(NumPar)) %>% #sum all the parentage observations at each DESTINATION
                    distinct(destination, .keep_all = T) %>%
                    select(-source) %>%
                    mutate(NumUnassigned=NumSimSampRec-NumPar)
                                
                
                #finally, join all of the sites surveyed for a complete parentage matrix
                #for this, need to have a source/destination column for all sites
                PropSamp2014 <- PropSamp2014 %>%
                                rename(source="site")
                
                PropSamp2014$destination <- PropSamp2014$source #make another column for destination
                
                #this below will become parentage matrix! 
                TotalSimPar2 <- full_join(TotalSimPar,PropSamp2014, by=c("source", "destination")) %>%
                    mutate(NumPar=ifelse(is.na(NumPar), 0, NumPar))
        
                
                #spread into matrix format
                SimDispMat <- TotalSimPar2 %>%
                    filter(source %in% NGenOffs2014$site) %>%
                    arrange(source, destination)  %>%
                    spread(destination, NumPar) %>%
                    select(-source)
                SimDispMat[is.na(SimDispMat)] <- 0
                
                #add in unassigned row
                unassigned <- t(as.matrix(SimSampSummary2014 %>% arrange(destination) %>% ungroup() %>% select(NumUnassigned)))
                rownames(unassigned)<-NULL
                colnames(unassigned)<-names(SimDispMat)
                
                SimDispMatFull2014 <- rbind(SimDispMat, unassigned)
                colnames(SimDispMatFull2014) <- NULL

        #The full remade parentage matrix
        Assignments2014 <- SimDispMatFull2014
    
    #NOW FIT THE KERNEL


x <- list(Distances=Distances, Assignments=Assignments2014, Sampled_reefs=Sampled_reefs2014, Reef_sizes=Reef_sizes, Adult_sample_proportions=Adult_sample_proportions2014) #put inputs into a list because that's the bbmle format

Sim2014Fit <- suppressWarnings(mle2(LL_kt_bbmle, start=list(k=-3, theta=1), lower=c(-10, 0.15), upper=c(10, 8), method="L-BFGS-B", data=x, control=list(maxit=500)))

BestK2014 <- as.numeric(coef(Sim2014Fit)[1])
BestTheta2014 <- as.numeric(coef(Sim2014Fit)[2])
MDD2014 <- as.numeric(cubintegrate(integrate_kernel_sum1, lower = 0, upper = Inf, k=BestK2014, theta=BestTheta2014, , method = "pcubature")$integral)
k_eval <- BestK2014
theta_eval <- BestTheta2014
Med2014  <- round(nleqslv(x = 7, fn = cdf_solve)$x, 2) 
Dist90_2014 <- round(nleqslv(x = 7, fn = cdf_solve90)$x, 2)
    
    #store the info in this df
    SimulatedKernels2014_beta <- as.data.frame(matrix(nrow=1, ncol=7), stringsAsFactors = FALSE)
    colnames(SimulatedKernels2014_beta) <- col
    
    SimulatedKernels2014_beta$year <- 2014
    SimulatedKernels2014_beta$k <- BestK2014
    SimulatedKernels2014_beta$theta <- BestTheta2014
    SimulatedKernels2014_beta$mdd <- MDD2014
    SimulatedKernels2014_beta$med <- Med2014
    SimulatedKernels2014_beta$dist90 <- Dist90_2014
    SimulatedKernels2014_beta$iteration <- n
    
    
#join results into larger df
SimulatedKernels2014 <- bind_rows(SimulatedKernels2014, SimulatedKernels2014_beta)
    
setTxtProgressBar(pb, n)


}
close(pb)
EndTime <- Sys.time()
EndTime-StartTime
options(warn=0) #turn warnings back on


write.csv(SimulatedKernels2014, file="~/oceanography/script_output/KernelFits/Adjust_Survival_NoRecentRecruits/1000SimulatedKernels2014SurvNoRecentRec.csv", row.names=F)



In [ ]:
#assemble all of the components for annual kernel fitting, search and replace for the years
NGenOffs2014 <- AnnualRecsSamp %>%
    filter(year=="2014")

nrow(NGenOffs2014) #should be 9 sites sampled for 2014

#***
PropSamp2014 <- PropSamp %>%  #add in all sampled sites as possible parent sites
                filter(end_year =="2014" & PropSamp > 0) %>%
                #rename(source="site") %>%
                select(-end_year, -PropSamp)

PropSamp2014 <- PropSamp2014 %>%
                rename(source="site")

#PropSamp2014$destination <- PropSamp2014$source #make another column for destination
#***
centroids <- read.csv("~/oceanography/empirical_data/site_centroids_simulation_kernels.csv", header=TRUE, stringsAsFactors = F)

Area <- read.csv("~/oceanography/empirical_data/site_area_header_nonsurveyed_simulation_kernels.csv", header=TRUE, stringsAsFactors = F) %>%
    arrange(site) %>%
    filter(site %!in% c("near_north_full1", "near_north_full2", "near_north_full3", "near_south_full1", "near_south_full2", "near_south_full3")) %>%
    mutate(kmsq=msq*10^-6) %>%
    select(kmsq)
Reef_sizes <- as.matrix(Area)

#give every site in the distance matrix of the simulation (even if we didn't sample there) an index number
AllSites <- centroids %>%
    select(site) %>%
    arrange(site)
nrow(AllSites) #should be 34x1
AllSites$index <- seq(from=1, to=34, by=1)

SampledSites2014 <- inner_join(PropSamp2014 %>% select(source), AllSites, by=c(source="site")) 

#check for correct number of rows
nrow(SampledSites2014)==nrow(PropSamp2014) #should be true
SampledSites2014Index <- t(as.matrix(SampledSites2014$index))
ncol(SampledSites2014Index)

Sampled_reefs2014 <- SampledSites2014Index

#proportion sampled matrix for kernel fitting

#for annual
Adult_sample_proportions2014 <- PropSamp %>%  #add in all sampled sites as possible parent sites
                filter(end_year =="2014" & PropSamp > 0) 

Adult_sample_proportions2014 <- as.matrix(Adult_sample_proportions2014$PropSamp)
nrow(Adult_sample_proportions2014) #should be 9 for 2014

#distance matrix using the centroids with combined Magbangon
### List of source locations
sites_source <- centroids

### List of destination locations
sites_dest <- centroids

dist_mat_m <- distm(sites_source[,c('lon','lat')], sites_source[,c('lon','lat')], fun=distVincentyEllipsoid)
Distances <- dist_mat_m*10^-3

Centroids <- centroids %>%
    select(-site)



In [ ]:
#assemble all of the components for annual kernel fitting, search and replace for the years
NGenOffs2013 <- AnnualRecsSamp %>%
    filter(year=="2013")

nrow(NGenOffs2013) #should be 9 sites sampled for 2013

#***
PropSamp2013 <- PropSamp %>%  #add in all sampled sites as possible parent sites
                filter(end_year =="2013" & PropSamp > 0) %>%
                #rename(source="site") %>%
                select(-end_year, -PropSamp)

PropSamp2013 <- PropSamp2013 %>%
                rename(source="site")


SampledSites2013 <- inner_join(PropSamp2013 %>% select(source), AllSites, by=c(source="site")) 

#check for correct number of rows
nrow(SampledSites2013)==nrow(PropSamp2013) #should be true
SampledSites2013Index <- t(as.matrix(SampledSites2013$index))
ncol(SampledSites2013Index)

Sampled_reefs2013 <- SampledSites2013Index

#proportion sampled matrix for kernel fitting

#for annual
Adult_sample_proportions2013 <- PropSamp %>%  #add in all sampled sites as possible parent sites
                filter(end_year =="2013" & PropSamp > 0) 

Adult_sample_proportions2013 <- as.matrix(Adult_sample_proportions2013$PropSamp)
nrow(Adult_sample_proportions2013) #should be 9 for 2013




In [ ]:
#assemble all of the components for annual kernel fitting, search and replace for the years
NGenOffs2012 <- AnnualRecsSamp %>%
    filter(year=="2012")

nrow(NGenOffs2012) #should be 9 sites sampled for 2012

#***
PropSamp2012 <- PropSamp %>%  #add in all sampled sites as possible parent sites
                filter(end_year =="2012" & PropSamp > 0) %>%
                #rename(source="site") %>%
                select(-end_year, -PropSamp)

PropSamp2012 <- PropSamp2012 %>%
                rename(source="site")


SampledSites2012 <- inner_join(PropSamp2012 %>% select(source), AllSites, by=c(source="site")) 

#check for correct number of rows
nrow(SampledSites2012)==nrow(PropSamp2012) #should be true
SampledSites2012Index <- t(as.matrix(SampledSites2012$index))
ncol(SampledSites2012Index)

Sampled_reefs2012 <- SampledSites2012Index

#proportion sampled matrix for kernel fitting

#for annual
Adult_sample_proportions2012 <- PropSamp %>%  #add in all sampled sites as possible parent sites
                filter(end_year =="2012" & PropSamp > 0) 

Adult_sample_proportions2012 <- as.matrix(Adult_sample_proportions2012$PropSamp)
nrow(Adult_sample_proportions2012) #should be 9 for 2012




In [ ]:
#combine simulations
SimulationAll <- bind_rows(Simulation2012, Simulation2013, Simulation2014)

#make a DF of all of the sampled fish over 2012-2014
AllYearsRecruits <- AnnualRecsSamp %>% distinct(site, .keep_all = T)
sum(AllYearsRecruits$n_offs_gen_all_years) #should be 394 for 2012-2014

In [ ]:
#what if we account for post-settlement mortality?
#make a df with weighted values for the days of the simluation oldest to most recent
SimDates <- SimulationAll %>%
    distinct(date) %>%
    mutate(DateSeq=row_number()) %>% #assign row numbers, so more recent dates are higher than older
    mutate(SurvivalWeightExp=exp(DateSeq)) %>% #apply the exponential function to get a survival column
    mutate(SurvivalWeightExp=SurvivalWeightExp/max(SurvivalWeightExp))%>%#normalize as a survival "probability"
    mutate(SurvivalWeightLin=(((0.25/2)/5)*DateSeq)) %>%#maybe a linear survival function (y=mx)is more appropriate (eyeball from Allison's Fig 3C because what the hell why not we're just thinking here)
    mutate(SurvivalWeightLin=SurvivalWeightLin/(max(SurvivalWeightLin)*1.1)) #normalize as a survival "probability"

SimulationAllSurv <- left_join(SimulationAll, SimDates, by="date") 

In [ ]:
#get together kernel fitting components for all years/seasonal

#assemble all of the components for annual kernel fitting
centroids <- read.csv("~/oceanography/empirical_data/site_centroids_simulation_kernels.csv", header=TRUE)

Area <- read.csv("~/oceanography/empirical_data/site_area_header_nonsurveyed_simulation_kernels.csv", header=TRUE) %>%
    arrange(site) %>%
    filter(site %!in% c("near_north_full1", "near_north_full2", "near_north_full3", "near_south_full1", "near_south_full2", "near_south_full3")) %>%
    mutate(kmsq=msq*10^-6) %>%
    select(kmsq)
Reef_sizes <- as.matrix(Area)

#give every site in the distance matrix of the simulation (even if we didn't sample there) an index number
AllSites <- centroids %>%
    select(site) %>%
    arrange(site)
nrow(AllSites) #should be 34x1
AllSites$index <- seq(from=1, to=34, by=1)

PropSampAll <- PropSamp %>%  #add in all sampled sites as possible parent sites
            filter(end_year =="2014" & PropSamp > 0) %>%
            rename(source="site") %>%
            select(-end_year, -PropSamp)

SampledSitesAll <- inner_join(PropSampAll %>% select(source), AllSites, by=c(source="site")) 

#check for correct number of rows
nrow(SampledSitesAll)==nrow(PropSampAll) #should be true
SampledSitesAllIndex <- t(as.matrix(SampledSitesAll$index))
ncol(SampledSitesAllIndex)

Sampled_reefsAll <- SampledSitesAllIndex

#proportion sampled matrix for kernel fitting

Adult_sample_proportionsAll <- PropSamp %>%  #add in all sampled sites as possible parent sites
                filter(end_year =="2014" & PropSamp > 0) 

Adult_sample_proportionsAll <- as.matrix(Adult_sample_proportionsAll$PropSamp)
nrow(Adult_sample_proportionsAll) 

#distance matrix using the centroids with combined Magbangon
### List of source locations
sites_source <- centroids

### List of destination locations
sites_dest <- centroids

dist_mat_m <- distm(sites_source[,c('lon','lat')], sites_source[,c('lon','lat')], fun=distVincentyEllipsoid)
Distances <- dist_mat_m*10^-3

Centroids <- centroids %>%
    select(-site)

#for the all years sites sampled in the simulation, it's a total of 16. That's lower than the all year used in kernel fitting because we didn't sample Sitio Tugas or Gabas until after 2014. We also combined the Magbangons, so 19-16=3 accounted for

In [ ]:
#All of the corresponding simulation years combined (2011-14)

col <- c("year", "k", "theta", "mdd", "med", "dist90", "iteration")
SimulatedKernelsAll <- as.data.frame(matrix(nrow=0, ncol=7), stringsAsFactors = FALSE)
colnames(SimulatedKernelsAll) <- col

#don't print warnings for this loop, they are only for setting row names in a tibble which is depracated. But it works. 
options(warn=-1)

pb <- txtProgressBar(min = 0, max = 1000, style = 3)#7 years in each interation

StartTime <- Sys.time()


for(n in 1:1000){
        
    #SAMPLE THE SIMULATION DATA, then fit a kernel
    
    SimulatedSamplingAll = SimulationAll[FALSE,]
    
    for(i in 1:nrow(AllYearsRecruits)){ 
        
        destination_eval <- as.character(AllYearsRecruits$site[i]) #pick out a destination site
        
        SimSampDestination <- SimulationAllSurv %>%
                                    filter(destination==destination_eval) %>%
                                    sample_n(as.numeric(AllYearsRecruits$n_offs_gen_all_years[i]), weight=SurvivalWeightLin, replace=F) %>% #sample particles that landed at that destination, number corresponding to the actual number sampled at that site in this year
                                    select(one_of(names(SimulationAll)))
        
        SimulatedSamplingAll <- bind_rows(SimulatedSamplingAll, SimSampDestination) #build into a sampled particle df
    }

    #assign a numeric ID for each row (which is a sampled particle)
    SimulatedSamplingAll <- SimulatedSamplingAll %>%
        mutate(ParticleSampID=paste ("P", row_number(), sep = "", collapse = NULL))
    
    #nrow(SimulatedSamplingAll)==sum(NGenOffsAll$n_offs_gen) #should be TRUE
    
    #now randomly assign parentage match status to some of these rows
    NumPar <- as.numeric(kernels %>% #should be 37 for 2012-2014
        filter(Year %in% c("2012", "2013", "2014")) %>%
        select(NumParentageMatches) %>%
        summarise(NumParentageMatches=sum(NumParentageMatches)))
    
    SimulatedSamplingAllPar <- SimulatedSamplingAll %>%
        ungroup() %>%
        filter(source %in% NGenOffsAll$site) %>% #we can only assign parentage if we sampled the source site, we've already ensured that we sampled the destination site in the for loop by indexing destination sites to evaluate 
        sample_n(NumPar, replace=F) %>% #sample number of rows consitent with numbers of parentage matches
        mutate(Parentage=1) #%>% #assign these rows a positive match value
        #arrange(date, source, destination) %>%#arrange by the same order as the orginal df, then bind columns together to avoid unexpected repeats from a join
        #select(date, source, destination, Parentage) #drop columns that will result in repeats 
    
    #find the rows that haven't been assigned parentage
    SimulatedSamplingAll_2 <- anti_join((SimulatedSamplingAll %>% ungroup()), SimulatedSamplingAllPar, by=c("ParticleSampID")) 
    
    SimulatedSamplingAll_2$Parentage <- 0 #add in the column for parentage
    
    #combine parentage and non-parentage dfs
    SimulatedSamplingAllPar3 <- bind_rows(SimulatedSamplingAllPar, SimulatedSamplingAll_2) %>%
        mutate(YearSampled=as.character(YearSampled))
    
    
    #NOW FORMAT INTO PARENTAGE MATRIX
    
                PropSampAll <- PropSamp %>%  #add in all sampled sites as possible parent sites
                            filter(end_year =="2014" & PropSamp > 0) %>%
                            #rename(source="site") %>%
                            select(-end_year, -PropSamp)

                #PropSampAll$destination <- PropSampAll$source #make another column for destination
                
                #make a table with the total number of particles sampled at each site
                TotalSimSamp <- SimulatedSamplingAllPar3 %>% 
                            select(source, destination) %>%
                            group_by(destination) %>%
                            summarise(NumSimSampRec=n()) %>%
                            ungroup()
               
                
                #add in all surveyed sites
                TotalSimSamp2 <- left_join(PropSampAll, TotalSimSamp, by=c(site="destination")) %>% #join sites to destination, which will add a row for the surveyed sites with no samples
                    mutate(NumSimSampRec=ifelse(is.na(NumSimSampRec), 0, NumSimSampRec)) %>% #change the Na's that result to 0s, because we didn't get samples there
                    rename(destination="site")#rename sites to destination after joining
                
                
                
                #make a table with the total number of parentage matches found in the simulated loop
                TotalSimPar <- SimulatedSamplingAllPar3 %>% 
                            select(source, destination, Parentage) %>%
                            group_by(source,destination) %>%
                            mutate(NumPar=sum(Parentage)) %>% #sum all the parentage observations along each ROUTE
                            select(-Parentage) %>%
                            ungroup() %>%
                            distinct(source, destination, .keep_all = T)
                
                
                #combine for a summary table BY DESTINATION, this below will be for UNASSIGNED ROWS
                SimSampSummaryAll <- left_join(TotalSimSamp2, TotalSimPar, by="destination") %>%
                    distinct(source, destination, .keep_all = T) %>%
                    group_by(destination) %>%
                    mutate(NumPar=ifelse(is.na(NumPar), 0, NumPar)) %>% #change the Na's that result to 0s, because we didn't get samples there
                    mutate(NumPar=sum(NumPar)) %>% #sum all the parentage observations at each DESTINATION
                    distinct(destination, .keep_all = T) %>%
                    select(-source) %>%
                    mutate(NumUnassigned=NumSimSampRec-NumPar)
                                
                
                #finally, join all of the sites surveyed for a complete parentage matrix
                #for this, need to have a source/destination column for all sites
                PropSampAll <- PropSampAll %>%
                                rename(source="site")
                
                PropSampAll$destination <- PropSampAll$source #make another column for destination
                
                #this below will become parentage matrix! 
                TotalSimPar2 <- full_join(TotalSimPar,PropSampAll, by=c("source", "destination")) %>%
                    mutate(NumPar=ifelse(is.na(NumPar), 0, NumPar))
        
                
                #spread into matrix format
                SimDispMat <- TotalSimPar2 %>%
                    filter(source %in% NGenOffsAll$site) %>%
                    arrange(source, destination)  %>%
                    spread(destination, NumPar) %>%
                    select(-source)
                SimDispMat[is.na(SimDispMat)] <- 0
                
                #add in unassigned row
                unassigned <- t(as.matrix(SimSampSummaryAll %>% arrange(destination) %>% ungroup() %>% select(NumUnassigned)))
                rownames(unassigned)<-NULL
                colnames(unassigned)<-names(SimDispMat)
                
                SimDispMatFullAll <- rbind(SimDispMat, unassigned)
                colnames(SimDispMatFullAll) <- NULL

        #The full remade parentage matrix
        AssignmentsAll <- SimDispMatFullAll
    
    #NOW FIT THE KERNEL


x <- list(Distances=Distances, Assignments=AssignmentsAll, Sampled_reefs=Sampled_reefsAll, Reef_sizes=Reef_sizes, Adult_sample_proportions=Adult_sample_proportionsAll) #put inputs into a list because that's the bbmle format

SimAllFit <- suppressWarnings(mle2(LL_kt_bbmle, start=list(k=-3, theta=1), lower=c(-10, 0.15), upper=c(10, 8), method="L-BFGS-B", data=x, control=list(maxit=500)))

BestKAll <- as.numeric(coef(SimAllFit)[1])
BestThetaAll <- as.numeric(coef(SimAllFit)[2])
MDDAll <- as.numeric(cubintegrate(integrate_kernel_sum1, lower = 0, upper = Inf, k=BestKAll, theta=BestThetaAll, , method = "pcubature")$integral)
k_eval <- BestKAll
theta_eval <- BestThetaAll
MedAll  <- round(nleqslv(x = 7, fn = cdf_solve)$x, 2) 
Dist90_All <- round(nleqslv(x = 7, fn = cdf_solve90)$x, 2)
    
    #store the info in this df
    SimulatedKernelsAll_beta <- as.data.frame(matrix(nrow=1, ncol=7), stringsAsFactors = FALSE)
    colnames(SimulatedKernelsAll_beta) <- col
    
    SimulatedKernelsAll_beta$year <- NA
    SimulatedKernelsAll_beta$k <- BestKAll
    SimulatedKernelsAll_beta$theta <- BestThetaAll
    SimulatedKernelsAll_beta$med <- MedAll
    SimulatedKernelsAll_beta$mdd <- MDDAll
    SimulatedKernelsAll_beta$dist90 <- Dist90_All
    SimulatedKernelsAll_beta$iteration <- n
    
    
#join results into larger df
SimulatedKernelsAll <- bind_rows(SimulatedKernelsAll, SimulatedKernelsAll_beta)
    
setTxtProgressBar(pb, n)


}
close(pb)
EndTime <- Sys.time()
EndTime-StartTime
options(warn=0) #turn warnings back on


write.csv(SimulatedKernelsAll, file="~/oceanography/script_output/KernelFits/Adjust_Survival_NoRecentRecruits/1000SimulatedKernelsAllSurvNoRecentRec.csv", row.names=F)

__Seasonal kernel fits__

In [ ]:
SimulationSWM <- SimulationAllSurv %>% 
    filter(SimMonsoon=="SWM")
SimulationNEM <- SimulationAllSurv %>%
    filter(SimMonsoon=="NEM")

In [ ]:
#SWM of the corresponding simulation years combined (2011-SWM)

col <- c("season", "k", "theta", "mdd", "med", "dist90", "iteration")
SimulatedKernelsSWM <- as.data.frame(matrix(nrow=0, ncol=7), stringsAsFactors = FALSE)
colnames(SimulatedKernelsSWM) <- col

#don't print warnings for this loop, they are only for setting row names in a tibble which is depracated. But it works. 
options(warn=-1)

pb <- txtProgressBar(min = 0, max = 1000, style = 3)

StartTime <- Sys.time()


for(n in 1:1000){
        
    #SAMPLE THE SIMULATION DATA, then fit a kernel
    
    SimulatedSamplingSWM = SimulationSWM[FALSE,]
    
    
    for(i in 1:nrow(AllYearsRecruits)){ 
        
        destination_eval <- as.character(AllYearsRecruits$site[i]) #pick out a destination site
        
        SimSampDestination <- SimulationSWM %>%
                                    filter(destination==destination_eval) %>%
                                    sample_n(as.numeric(AllYearsRecruits$n_offs_gen_all_years[i]), weight=SurvivalWeightLin, replace=F) %>% #sample particles that landed at that destination, number corresponding to the actual number sampled at that site in this year
                                    select(one_of(names(SimulationAll)))
        
        SimulatedSamplingSWM <- bind_rows(SimulatedSamplingSWM, SimSampDestination) #build into a sampled particle df
    }

    #assign a numeric ID for each row (which is a sampled particle)
    SimulatedSamplingSWM <- SimulatedSamplingSWM %>%
        mutate(ParticleSampID=paste ("P", row_number(), sep = "", collapse = NULL))
    
    #nrow(SimulatedSamplingSWM)==sum(NGenOffsSWM$n_offs_gen) #should be TRUE
    
   #now randomly assign parentage match status to some of these rows
    NumPar <- as.numeric(kernels %>% #should be 37 for 2012-2014
        filter(Year %in% c("2012", "2013", "2014")) %>%
        select(NumParentageMatches) %>%
        summarise(NumParentageMatches=sum(NumParentageMatches)))
    
    SimulatedSamplingSWMPar <- SimulatedSamplingSWM %>%
        ungroup() %>%
        filter(source %in% AllYearsRecruits$site) %>% #we can only assign parentage if we sampled the source site, we've already ensured that we sampled the destination site in the for loop by indexing destination sites to evaluate 
        sample_n(NumPar, replace=F) %>% #sample number of rows consitent with numbers of parentage matches
        mutate(Parentage=1) #%>% #assign these rows a positive match value
        #arrange(date, source, destination) %>%#arrange by the same order as the orginal df, then bind columns together to avoid unexpected repeats from a join
        #select(date, source, destination, Parentage) #drop columns that will result in repeats 
    
    #find the rows that haven't been assigned parentage
    SimulatedSamplingSWM_2 <- anti_join((SimulatedSamplingSWM %>% ungroup()), SimulatedSamplingSWMPar, by=c("ParticleSampID")) 
    
    SimulatedSamplingSWM_2$Parentage <- 0 #add in the column for parentage
    
    #combine parentage and non-parentage dfs
    SimulatedSamplingSWMPar3 <- bind_rows(SimulatedSamplingSWMPar, SimulatedSamplingSWM_2) %>%
        mutate(YearSampled=as.character(YearSampled))
    
    
    #NOW FORMAT INTO PARENTAGE MATRIX
    
                PropSampSWM <- PropSamp %>%  #add in SWM sampled sites as possible parent sites
                            filter(end_year =="2014" & PropSamp > 0) %>%
                            #rename(source="site") %>%
                            select(-end_year, -PropSamp)

                #PropSampSWM$destination <- PropSampSWM$source #make another column for destination
                
                #make a table with the total number of particles sampled at each site
                TotalSimSamp <- SimulatedSamplingSWMPar3 %>% 
                            select(source, destination) %>%
                            group_by(destination) %>%
                            summarise(NumSimSampRec=n()) %>%
                            ungroup()
               
                
                #add in SWM surveyed sites
                TotalSimSamp2 <- left_join(PropSampSWM, TotalSimSamp, by=c(site="destination")) %>% #join sites to destination, which will add a row for the surveyed sites with no samples
                    mutate(NumSimSampRec=ifelse(is.na(NumSimSampRec), 0, NumSimSampRec)) %>% #change the Na's that result to 0s, because we didn't get samples there
                    rename(destination="site")#rename sites to destination after joining
                
                
                
                #make a table with the total number of parentage matches found in the simulated loop
                TotalSimPar <- SimulatedSamplingSWMPar3 %>% 
                            select(source, destination, Parentage) %>%
                            group_by(source,destination) %>%
                            mutate(NumPar=sum(Parentage)) %>% #sum SWM the parentage observations along each ROUTE
                            select(-Parentage) %>%
                            ungroup() %>%
                            distinct(source, destination, .keep_all = T)
                
                
                #combine for a summary table BY DESTINATION, this below will be for UNASSIGNED ROWS
                SimSampSummarySWM <- left_join(TotalSimSamp2, TotalSimPar, by="destination") %>%
                    distinct(source, destination, .keep_all = T) %>%
                    group_by(destination) %>%
                    mutate(NumPar=ifelse(is.na(NumPar), 0, NumPar)) %>% #change the Na's that result to 0s, because we didn't get samples there
                    mutate(NumPar=sum(NumPar)) %>% #sum SWM the parentage observations at each DESTINATION
                    distinct(destination, .keep_all = T) %>%
                    select(-source) %>%
                    mutate(NumUnassigned=NumSimSampRec-NumPar)
                                
                
                #finSWMy, join SWM of the sites surveyed for a complete parentage matrix
                #for this, need to have a source/destination column for SWM sites
                PropSampSWM <- PropSampSWM %>%
                                rename(source="site")
                
                PropSampSWM$destination <- PropSampSWM$source #make another column for destination
                
                #this below will become parentage matrix! 
                TotalSimPar2 <- full_join(TotalSimPar,PropSampSWM, by=c("source", "destination")) %>%
                    mutate(NumPar=ifelse(is.na(NumPar), 0, NumPar))
        
                
                #spread into matrix format
                SimDispMat <- TotalSimPar2 %>%
                    filter(source %in% AllYearsRecruits$site) %>%
                    arrange(source, destination)  %>%
                    spread(destination, NumPar) %>%
                    select(-source)
                SimDispMat[is.na(SimDispMat)] <- 0
                
                #add in unassigned row
                unassigned <- t(as.matrix(SimSampSummarySWM %>% arrange(destination) %>% ungroup() %>% select(NumUnassigned)))
                rownames(unassigned)<-NULL
                colnames(unassigned)<-names(SimDispMat)
                
                SimDispMatFullSWM <- rbind(SimDispMat, unassigned)
                colnames(SimDispMatFullSWM) <- NULL

        #The full remade parentage matrix
        AssignmentsSWM <- SimDispMatFullSWM
    
    #NOW FIT THE KERNEL


x <- list(Distances=Distances, Assignments=AssignmentsSWM, Sampled_reefs=Sampled_reefsAll, Reef_sizes=Reef_sizes, Adult_sample_proportions=Adult_sample_proportionsAll) #put inputs into a list because that's the bbmle format

SimSWMFit <- suppressWarnings(mle2(LL_kt_bbmle, start=list(k=-3, theta=1), lower=c(-10, 0.15), upper=c(10, 8), method="L-BFGS-B", data=x, control=list(maxit=500)))

BestKSWM <- as.numeric(coef(SimSWMFit)[1])
BestThetaSWM <- as.numeric(coef(SimSWMFit)[2])
MDDSWM <- as.numeric(cubintegrate(integrate_kernel_sum1, lower = 0, upper = Inf, k=BestKSWM, theta=BestThetaSWM, , method = "pcubature")$integral)
k_eval <- BestKSWM
theta_eval <- BestThetaSWM
MedSWM  <- round(nleqslv(x = 7, fn = cdf_solve)$x, 2) 
Dist90_SWM <- round(nleqslv(x = 7, fn = cdf_solve90)$x, 2)
    
    #store the info in this df
    SimulatedKernelsSWM_beta <- as.data.frame(matrix(nrow=1, ncol=7), stringsAsFactors = FALSE)
    colnames(SimulatedKernelsSWM_beta) <- col
    
    SimulatedKernelsSWM_beta$season <- NA
    SimulatedKernelsSWM_beta$k <- BestKSWM
    SimulatedKernelsSWM_beta$theta <- BestThetaSWM
    SimulatedKernelsSWM_beta$med <- MedSWM
    SimulatedKernelsSWM_beta$mdd <- MDDSWM
    SimulatedKernelsSWM_beta$dist90 <- Dist90_SWM
    SimulatedKernelsSWM_beta$iteration <- n

    
#join results into larger df
SimulatedKernelsSWM <- bind_rows(SimulatedKernelsSWM, SimulatedKernelsSWM_beta) %>%
    mutate(season="SWM")
    
setTxtProgressBar(pb, n)


}
close(pb)
EndTime <- Sys.time()
EndTime-StartTime
options(warn=0) #turn warnings back on


write.csv(SimulatedKernelsSWM, file="~/oceanography/script_output/KernelFits/Adjust_Survival_NoRecentRecruits/1000SimulatedKernelsSWMSurvNoRecentRec.csv", row.names=F)

In [ ]:
#NEM of the corresponding simulation years combined (2011-NEM)

col <- c("season", "k", "theta", "mdd", "med", "dist90", "iteration")
SimulatedKernelsNEM <- as.data.frame(matrix(nrow=0, ncol=7), stringsAsFactors = FALSE)
colnames(SimulatedKernelsNEM) <- col

#don't print warnings for this loop, they are only for setting row names in a tibble which is depracated. But it works. 
options(warn=-1)

pb <- txtProgressBar(min = 0, max = 1000, style = 3)

StartTime <- Sys.time()


for(n in 1:1000){
        
    #SAMPLE THE SIMULATION DATA, then fit a kernel
    
    SimulatedSamplingNEM = SimulationNEM[FALSE,]
    
    
    for(i in 1:nrow(AllYearsRecruits)){ 
        
        destination_eval <- as.character(AllYearsRecruits$site[i]) #pick out a destination site
        
        SimSampDestination <- SimulationNEM %>%
                                    filter(destination==destination_eval) %>%
                                    sample_n(as.numeric(AllYearsRecruits$n_offs_gen_all_years[i]), weight=SurvivalWeightLin, replace=F) %>% #sample particles that landed at that destination, number corresponding to the actual number sampled at that site in this year
                                    select(one_of(names(SimulationAll)))
        
        SimulatedSamplingNEM <- bind_rows(SimulatedSamplingNEM, SimSampDestination) #build into a sampled particle df
    }

    #assign a numeric ID for each row (which is a sampled particle)
    SimulatedSamplingNEM <- SimulatedSamplingNEM %>%
        mutate(ParticleSampID=paste ("P", row_number(), sep = "", collapse = NULL))
    
    #nrow(SimulatedSamplingNEM)==sum(NGenOffsNEM$n_offs_gen) #should be TRUE
    
   #now randomly assign parentage match status to some of these rows
    NumPar <- as.numeric(kernels %>% #should be 37 for 2012-2014
        filter(Year %in% c("2012", "2013", "2014")) %>%
        select(NumParentageMatches) %>%
        summarise(NumParentageMatches=sum(NumParentageMatches)))
    
    SimulatedSamplingNEMPar <- SimulatedSamplingNEM %>%
        ungroup() %>%
        filter(source %in% AllYearsRecruits$site) %>% #we can only assign parentage if we sampled the source site, we've already ensured that we sampled the destination site in the for loop by indexing destination sites to evaluate 
        sample_n(NumPar, replace=F) %>% #sample number of rows consitent with numbers of parentage matches
        mutate(Parentage=1) #%>% #assign these rows a positive match value
        #arrange(date, source, destination) %>%#arrange by the same order as the orginal df, then bind columns together to avoid unexpected repeats from a join
        #select(date, source, destination, Parentage) #drop columns that will result in repeats 
    
    #find the rows that haven't been assigned parentage
    SimulatedSamplingNEM_2 <- anti_join((SimulatedSamplingNEM %>% ungroup()), SimulatedSamplingNEMPar, by=c("ParticleSampID")) 
    
    SimulatedSamplingNEM_2$Parentage <- 0 #add in the column for parentage
    
    #combine parentage and non-parentage dfs
    SimulatedSamplingNEMPar3 <- bind_rows(SimulatedSamplingNEMPar, SimulatedSamplingNEM_2) %>%
        mutate(YearSampled=as.character(YearSampled))
    
    
    #NOW FORMAT INTO PARENTAGE MATRIX
    
                PropSampNEM <- PropSamp %>%  #add in NEM sampled sites as possible parent sites
                            filter(end_year =="2014" & PropSamp > 0) %>%
                            #rename(source="site") %>%
                            select(-end_year, -PropSamp)

                #PropSampNEM$destination <- PropSampNEM$source #make another column for destination
                
                #make a table with the total number of particles sampled at each site
                TotalSimSamp <- SimulatedSamplingNEMPar3 %>% 
                            select(source, destination) %>%
                            group_by(destination) %>%
                            summarise(NumSimSampRec=n()) %>%
                            ungroup()
               
                
                #add in NEM surveyed sites
                TotalSimSamp2 <- left_join(PropSampNEM, TotalSimSamp, by=c(site="destination")) %>% #join sites to destination, which will add a row for the surveyed sites with no samples
                    mutate(NumSimSampRec=ifelse(is.na(NumSimSampRec), 0, NumSimSampRec)) %>% #change the Na's that result to 0s, because we didn't get samples there
                    rename(destination="site")#rename sites to destination after joining
                
                
                
                #make a table with the total number of parentage matches found in the simulated loop
                TotalSimPar <- SimulatedSamplingNEMPar3 %>% 
                            select(source, destination, Parentage) %>%
                            group_by(source,destination) %>%
                            mutate(NumPar=sum(Parentage)) %>% #sum NEM the parentage observations along each ROUTE
                            select(-Parentage) %>%
                            ungroup() %>%
                            distinct(source, destination, .keep_all = T)
                
                
                #combine for a summary table BY DESTINATION, this below will be for UNASSIGNED ROWS
                SimSampSummaryNEM <- left_join(TotalSimSamp2, TotalSimPar, by="destination") %>%
                    distinct(source, destination, .keep_all = T) %>%
                    group_by(destination) %>%
                    mutate(NumPar=ifelse(is.na(NumPar), 0, NumPar)) %>% #change the Na's that result to 0s, because we didn't get samples there
                    mutate(NumPar=sum(NumPar)) %>% #sum NEM the parentage observations at each DESTINATION
                    distinct(destination, .keep_all = T) %>%
                    select(-source) %>%
                    mutate(NumUnassigned=NumSimSampRec-NumPar)
                                
                
                #finNEMy, join NEM of the sites surveyed for a complete parentage matrix
                #for this, need to have a source/destination column for NEM sites
                PropSampNEM <- PropSampNEM %>%
                                rename(source="site")
                
                PropSampNEM$destination <- PropSampNEM$source #make another column for destination
                
                #this below will become parentage matrix! 
                TotalSimPar2 <- full_join(TotalSimPar,PropSampNEM, by=c("source", "destination")) %>%
                    mutate(NumPar=ifelse(is.na(NumPar), 0, NumPar))
        
                
                #spread into matrix format
                SimDispMat <- TotalSimPar2 %>%
                    filter(source %in% AllYearsRecruits$site) %>%
                    arrange(source, destination)  %>%
                    spread(destination, NumPar) %>%
                    select(-source)
                SimDispMat[is.na(SimDispMat)] <- 0
                
                #add in unassigned row
                unassigned <- t(as.matrix(SimSampSummaryNEM %>% arrange(destination) %>% ungroup() %>% select(NumUnassigned)))
                rownames(unassigned)<-NULL
                colnames(unassigned)<-names(SimDispMat)
                
                SimDispMatFullNEM <- rbind(SimDispMat, unassigned)
                colnames(SimDispMatFullNEM) <- NULL

        #The full remade parentage matrix
        AssignmentsNEM <- SimDispMatFullNEM
    
    #NOW FIT THE KERNEL


x <- list(Distances=Distances, Assignments=AssignmentsNEM, Sampled_reefs=Sampled_reefsAll, Reef_sizes=Reef_sizes, Adult_sample_proportions=Adult_sample_proportionsAll) #put inputs into a list because that's the bbmle format

SimNEMFit <- suppressWarnings(mle2(LL_kt_bbmle, start=list(k=-3, theta=1), lower=c(-10, 0.15), upper=c(10, 8), method="L-BFGS-B", data=x, control=list(maxit=500)))

BestKNEM <- as.numeric(coef(SimNEMFit)[1])
BestThetaNEM <- as.numeric(coef(SimNEMFit)[2])
MDDNEM <- as.numeric(cubintegrate(integrate_kernel_sum1, lower = 0, upper = Inf, k=BestKNEM, theta=BestThetaNEM, , method = "pcubature")$integral)
k_eval <- BestKNEM
theta_eval <- BestThetaNEM
MedNEM  <- round(nleqslv(x = 7, fn = cdf_solve)$x, 2) 
Dist90_NEM <- round(nleqslv(x = 7, fn = cdf_solve90)$x, 2)
    
    #store the info in this df
    SimulatedKernelsNEM_beta <- as.data.frame(matrix(nrow=1, ncol=7), stringsAsFactors = FALSE)
    colnames(SimulatedKernelsNEM_beta) <- col
    
    SimulatedKernelsNEM_beta$season <- NA
    SimulatedKernelsNEM_beta$k <- BestKNEM
    SimulatedKernelsNEM_beta$theta <- BestThetaNEM
    SimulatedKernelsNEM_beta$med <- MedNEM
    SimulatedKernelsNEM_beta$mdd <- MDDNEM
    SimulatedKernelsNEM_beta$dist90 <- Dist90_NEM
    SimulatedKernelsNEM_beta$iteration <- n

    
#join results into larger df
SimulatedKernelsNEM <- bind_rows(SimulatedKernelsNEM, SimulatedKernelsNEM_beta) %>%
    mutate(season="NEM")
    
setTxtProgressBar(pb, n)


}
close(pb)
EndTime <- Sys.time()
EndTime-StartTime
options(warn=0) #turn warnings back on


write.csv(SimulatedKernelsNEM, file="~/oceanography/script_output/KernelFits/Adjust_Survival_NoRecentRecruits/1000SimulatedKernelsNEMSurvNoRecentRec.csv", row.names=F)

In [ ]:
head(SimulatedKernels2012)
head(SimulatedKernels2013)
head(SimulatedKernels2014)
head(SimulatedKernelsAll)
head(SimulatedKernelsNEM)
head(SimulatedKernelsSWM)